In [3]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Configuration
LATENT_DIM = 128          # adapte si besoin
NUM_IMAGES = 4            # par ton
OUTPUT_DIR = "generated"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Tons (5 classes)
CLUSTERS = {
    "chaud_clair": 0,
    "chaud_pastel": 1,
    "froid_clair": 2,
    "froid_pastel": 3,
    "neutre": 4
}

In [4]:
# Charger le décodeur sauvegardé
decoder = load_model("decoder.h5")  # adapte le chemin

def generate_images(decoder, tone_name, tone_id, n):
    z = np.random.normal(size=(n, LATENT_DIM))
    labels = np.full((n, 1), tone_id, dtype=np.int32)

    images = decoder.predict([z, labels], verbose=0)

    for i, img in enumerate(images):
        img = (img * 255).astype(np.uint8)
        Image.fromarray(img).save(
            os.path.join(OUTPUT_DIR, f"{tone_name}_{i+1}.png")
        )

# Génération pour tous les tons
for tone, idx in CLUSTERS.items():
    generate_images(decoder, tone, idx, NUM_IMAGES)

In [5]:
def make_gif(image_dir, gif_name="tones.gif", duration=500):
    paths = sorted([
        os.path.join(image_dir, f)
        for f in os.listdir(image_dir)
        if f.endswith(".png")
    ])

    images = [Image.open(p) for p in paths]

    images[0].save(
        os.path.join(image_dir, gif_name),
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0
    )

# Créer le GIF
make_gif(OUTPUT_DIR, gif_name="tones.gif", duration=500)